In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_cirrhosis = pd.read_csv(r'C:\Users\mpola\OneDrive\Desktop\Career\Proje\Liver Cirrhosis Prediction\data\cirrhosis.csv')

### Exploratory Data Analysis

In [ ]:
from scipy.stats import skew

# To get a better idea of the dataset we are working with, it makes sense to use some functions to analyze the columns we have.


# Padding the column names with spaces at the end so they look nicer once printed
column_names = list(df_cirrhosis.columns)
max_length = max(len(name) for name in column_names)

formatted_names = []
for name in column_names:
    formatted_names.append(name + " " * (max_length - len(name)))


# The skewness calculation shows that most columns' data is heavily skewed, so the distribution is not close to normal.
# This makes using mean values to fill in NaN entries a poor choice, so we will use median instead.
print('---Skewness Values of Numerical Columns---')
print('| Column Name | Skewness |')
print('|-------------|----------|')
for index, col in enumerate(df_cirrhosis.columns):
    if df_cirrhosis[col].dtype == 'float64' or df_cirrhosis[col].dtype == 'int64': 
        skewness = skew(df_cirrhosis[col], nan_policy='omit')
        print(f'| {formatted_names[index]} | {skewness} |')

# print('\n\n---Dataframe Head---')
# print(df_cirrhosis.head(20))

print('\n\n---Dataframe Info---')
print(df_cirrhosis.info())

In [ ]:
print('\n\n---Dataframe Description---')
print(df_cirrhosis.describe())

The max values for Cholesterol, Copper, Alk_Phos, Bilirubin and Tryglicerides columns  are anywhere between roughly 4 to 8 times higher than the 75% values, so there are plenty of outliers here. With the exception of Tryglicerides, these columns also have very high standard deviation values compared to their means, so the data seems to be very irregular in general.

In [ ]:
# Defining the lists that contain numerical and categorical columns to use later

numerical_cols = df_cirrhosis.select_dtypes(include=np.number).columns
categorical_cols = df_cirrhosis.select_dtypes(include=('object')).columns

In [ ]:
sns.pairplot(df_cirrhosis[numerical_cols])

plt.show()

At a first glance at the pair plots of our numerical data, there aren't any strong correlations or clustering to take into immediate consideration. It may seem like the inclusion of the ID column is an oversight, but looking closely, we can see that there is a very clear cut-off line for the N-Days column where its upper bound somewhat linearly decreases with ID until roughly the 300 ID mark, but then it resets and starts linearly decreasing again, and past this 300-ID line we have no data for some of the columns like Cholesterol, Copper and Alk_Phos. This is in line with the dataset description mentioning that the first 312 participants contain more complete data whereas the remaining 106 did not participate in the clinical trial of the drug D-penicillamine, only giving consent to basic measurements. This can cause our predictions to be biased or be performed with incomplete information, but given the size of the dataset, I chose not to exclude the last 106 participants.

In [ ]:
corr = df_cirrhosis[numerical_cols].corr(method = 'spearman')
plt.figure(figsize = (15, 15), dpi = 300)
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr, mask = mask, annot = True, annot_kws = {'size' : 15})

plt.show()

This dataset has plenty of outliers and some non-normal distributed data, so when testing for correlation I elected to check the Spearman correlation coefficient. The N_Days feature has a somewhat positive correlation with Albumin, and negative correlations with Bilirubin, Copper and cirrhosis stage, but these correlations are not strong enough to consider dimensionality reduction solely based off of them.

In [ ]:
df_cirrhosis['Stage'].value_counts()

As we will be trying to predict the stage of cirrhosis using other info and the count of each stage is imbalanced, using log loss to evaluate model performance is our best approach.

### Data Preprocesing

In [ ]:
# There are both numerical and categorical data in this dataset, with a lot of blank entries and outliers. In order to
# avoid problems with the prediction, we will be filling in the missing numerical data with the median values (as explained before) 
# and the missing categorical data with the mode values

for col in numerical_cols:
    # Using median of numerical columns to replace NaN values since this dataset contains outliers
    df_cirrhosis[col] = df_cirrhosis[col].fillna(df_cirrhosis[col].median())

for col in categorical_cols:
    # Using mode of categorical columns to replace NaN values
    df_cirrhosis[col] = df_cirrhosis[col].fillna(df_cirrhosis[col].mode()[0])


In [ ]:
#########################################################
def detect_outliers_zscore(data, threshold = 3):
# In order to handle the outliers, we can check the entries' z-scores to first identify them.
# A custom function is defined to expedite this proccess.
    z = np.abs((data - np.mean(data)) / np.std(data))
    outliers = np.where(z > threshold)
    return outliers
#########################################################

# Capping the outlier values to minimize the influence of extreme values. Dropping rows is not a good idea given that we have
# a lot of columns and very few rows, and most rows have a few NaN values somewhere along them so dropping would leave us with
# practically zero data to work with.

for col in numerical_cols:
    outliers = detect_outliers_zscore(df_cirrhosis[col], 3)

    upper_bound = df_cirrhosis[col].mean() + 3 * df_cirrhosis[col].std()
    lower_bound = df_cirrhosis[col].mean() - 3 * df_cirrhosis[col].std()
    
    df_cirrhosis[col] = np.clip(df_cirrhosis[col], lower_bound, upper_bound)

In [ ]:
# Turning the stage feature into an integer for our models to handle better

# df_cirrhosis['Stage'] = df_cirrhosis['Stage'].astype(str) 
df_cirrhosis['Stage'] = df_cirrhosis['Stage'].astype(int) - 1

### Model Training

In [ ]:
# importing libraries to use: Our initial approach is to test out a wide variety of algorithms and ensemble methods
# with a moderate fold count to see which ones work best with our dataset. Afterwards, we can retry the top-performing
# models with incrementing fold counts until we find the sweet spot of performance and accuracy.

from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.metrics import log_loss, recall_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier

from category_encoders import OneHotEncoder

In [ ]:
#########################################################
def cross_validation(X, y, estimator, cv, label = ''):
# Since we want this test to be robust, it makes sense to define a custom function to expedite some of the steps.
# This function iterates through the folds of our stratified k-fold object with the given estimator,
# calculates the log-loss score and returns the log-loss scores list, predictions list and a print of these alongside the model name
    
    # Initiating the prediction arrays and score lists to store the values for each fold
    predictions_list = []
    log_loss_list = []
    recall_list = []
    
    for fold, (train_index, test_index) in enumerate(cv.split(X, y)):

        # We clone the model instead of working on the original to avoid data bleedthrough
        model = clone(estimator)
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train)
        
        # Clipping the values to exclude 0 and 1 to avoid issues with the log-loss function
        probabilities = model.predict_proba(X_test).clip(1e-15, 1 - 1e-15)
        predictions_list.append(probabilities)
        
        log_loss_score = log_loss(y_test, probabilities)
        log_loss_list.append(log_loss_score)

        # Predicted classes for recall calculation
        predicted_classes = model.predict(X_test) 
        
        recall = recall_score(y_test, predicted_classes, average='macro')
        recall_list.append(recall)
    
    print(f'| {label} | {np.mean(log_loss_list):.2f} ± {np.std(log_loss_list):.2f} | {np.mean(recall_list):.2f} ± {np.std(recall_list):.2f} |')
    
    return log_loss_list, predictions_list, recall_list
#########################################################

In [ ]:
# Initializing the models

seed = 221192

split = 10

#########################################################
def init_classifiers():
# This function is defined to initialize our models. It is not really needed as we clone the models in the cross_validation
# function and there is no reason to re-initialize them as long as we work with clones, but it is still nice to
# have all the models we're going to work with in an organized spot like this.
    
    rf_classifier = RandomForestClassifier(random_state=seed, class_weight='balanced_subsample') # Random Forest

    xgb_classifier = XGBClassifier(random_state=seed) # Gradient Boosting

    svm_classifier = SVC(random_state=seed, probability=True)  # Support Vector Machine

    lr_classifier = LogisticRegression(penalty='l2', C=1.0, solver='liblinear', random_state=seed, max_iter=5000) # Logistic Regression

    knn_classifier = KNeighborsClassifier() # K-Nearest Neighbors

    dt_classifier = DecisionTreeClassifier(random_state=seed) # Decision Tree

    ada_classifier = AdaBoostClassifier(random_state=seed) # AdaBoost classifier

    gnb_classifier = GaussianNB() # Gaussian Naive Bayes classifier
    
    return rf_classifier, xgb_classifier, svm_classifier, lr_classifier, knn_classifier, dt_classifier, ada_classifier, gnb_classifier
#########################################################

rf_classifier, xgb_classifier, svm_classifier, lr_classifier, knn_classifier, dt_classifier, ada_classifier, gnb_classifier = init_classifiers()

estimators = [
    ('rf', rf_classifier), 
    ('xgb', xgb_classifier), 
    ('svm', svm_classifier), 
    ('lr', lr_classifier), 
    ('knn', knn_classifier), 
    ('dt', dt_classifier),
    ('ada', ada_classifier),
    ('gnb', gnb_classifier)
]

In [ ]:
# Defining our random seed and split count for the stratified k-fold object, which then immediately
# gets fed into our cross_validation function

skf = StratifiedKFold(n_splits = split, 
                      shuffle = True, 
                      random_state = seed
                     )  


# Defining the dataframes that'll store each model's score and predictions
log_loss_frame, preds_frame, recall_frame = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

X = df_cirrhosis.drop('Stage', axis=1)
y = df_cirrhosis['Stage']

print(f'| Model | Log Loss (Mean ± Std) | Recall (Mean ± Std) |')
print(f'|-------|-----------------------|---------------------|')

# Running the cross_validation function for each model in our estimators list
for (label, model) in estimators:
    log_loss_frame[label], preds_frame[label], recall_frame[label] = cross_validation(X, y,
        make_pipeline(OneHotEncoder(cols = categorical_cols), 
                      StandardScaler(), 
                      PowerTransformer(), 
                      model),
        cv = skf,
        label = label
    )

In [ ]:
# This loop uses the stacking classifier method with our previously established methods, with the meta-learner
# changing on each iteration.

skf = StratifiedKFold(n_splits = split, 
                      shuffle = True, 
                      random_state = seed
                     )  


log_loss_frame_stacked, preds_frame_stacked, recall_frame_stacked = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

X = df_cirrhosis.drop('Stage', axis=1)
y = df_cirrhosis['Stage']

print(f'| Meta-Learner | Log Loss (Mean ± Std) | Recall (Mean ± Std) |')
print(f'|--------------|-----------------------|---------------------|')

for i, (label, meta_learner) in enumerate(estimators):

    # Creating a new list of our models to exclude the meta learner from the initial learners.
    initial_learners = []
    for j, (label2, estimator) in enumerate(estimators):
        if i != j:
            initial_learners.append((label2, estimator))
            
    stacking_classifier = StackingClassifier(estimators = initial_learners, 
                                             final_estimator = meta_learner 
                                            )
    
    log_loss_frame_stacked[label], preds_frame_stacked[label], recall_frame_stacked[label] = cross_validation(X, y,
        make_pipeline(OneHotEncoder(cols = categorical_cols), 
                      StandardScaler(), 
                      PowerTransformer(), 
                      stacking_classifier),
        cv = skf,
        label = label
    )

### Model Evaluation

In the end, we have the following log loss and recall values for our models:

| Model | Log Loss (Mean ± Std) | Recall (Mean ± Std) |
|-------|-----------------------|---------------------|
| rf | 1.16 ± 0.26 | 0.36 ± 0.05 |
| xgb | 1.54 ± 0.26 | 0.39 ± 0.05 |
| svm | 1.08 ± 0.06 | 0.35 ± 0.04 |
| lr | 1.12 ± 0.13 | 0.38 ± 0.05 |
| knn | 5.32 ± 1.07 | 0.37 ± 0.09 |
| dt | 19.74 ± 2.21 | 0.35 ± 0.09 |
| ada | 1.36 ± 0.00 | 0.38 ± 0.04 |
| gnb | 23.91 ± 2.91 | 0.31 ± 0.05 |

| Meta-Learner | Log Loss (Mean ± Std) | Recall (Mean ± Std) |
|--------------|-----------------------|---------------------|
| rf | 1.33 ± 0.49 | 0.35 ± 0.07 |
| xgb | 1.68 ± 0.22 | 0.36 ± 0.09 |
| svm | 1.11 ± 0.05 | 0.35 ± 0.03 |
| lr | 1.10 ± 0.08 | 0.34 ± 0.03 |
| knn | 6.34 ± 1.58 | 0.34 ± 0.06 |
| dt | 21.74 ± 2.11 | 0.28 ± 0.05 |
| ada | 1.36 ± 0.00 | 0.35 ± 0.05 |
| gnb | 4.68 ± 1.15 | 0.49 ± 0.09 |

In [ ]:
# Visualizing the log loss and recall values for our tests

fig, axes = plt.subplots(2, 2, figsize=(16, 12)) 

### Model Plots
log_loss_mean_model = log_loss_frame.mean()
log_loss_std_model = log_loss_frame.std()
sorted_models_log_loss = log_loss_mean_model.sort_values().index

recall_mean_model = recall_frame.mean()
recall_std_model = recall_frame.std()
sorted_models_recall = recall_mean_model.sort_values(ascending=False).index

# Log Loss Plot

sns.barplot(ax= axes[0,0],
            x = log_loss_mean_model[sorted_models_log_loss],
            y = sorted_models_log_loss,
            color='skyblue')

axes[0,0].errorbar(x = log_loss_mean_model[sorted_models_log_loss],
                 y   = sorted_models_log_loss,
                 xerr= log_loss_std_model[sorted_models_log_loss],
                 fmt='o',
                 color='black',
                 capsize=5)

axes[0,0].set_xlabel("Mean Log Loss")
axes[0,0].set_ylabel("Models")
axes[0,0].set_title("Mean Log Loss of Different Models")

# Recall Plot

sns.barplot(ax= axes[0,1],
            x = recall_mean_model[sorted_models_recall],
            y = sorted_models_recall,
            color='lightgreen')

axes[0,1].errorbar(x = recall_mean_model[sorted_models_recall],
                 y   = sorted_models_recall,
                 xerr= recall_std_model[sorted_models_recall],
                 fmt='o', color='black', capsize=5)

axes[0,1].set_xlabel("Mean Recall")
axes[0,1].set_ylabel("Models")
axes[0,1].set_title("Mean Recall of Different Models")


### Meta Learner Plots
log_loss_mean_stacked= log_loss_frame_stacked.mean()
log_loss_std_stacked = log_loss_frame_stacked.std()
sorted_meta_log_loss = log_loss_mean_stacked.sort_values().index

recall_mean_stacked= recall_frame_stacked.mean()
recall_std_stacked = recall_frame_stacked.std()
sorted_meta_recall = recall_mean_stacked.sort_values(ascending=False).index

# Log Loss Plot

sns.barplot(ax= axes[1,0],
            x = log_loss_mean_stacked[sorted_meta_log_loss],
            y = sorted_meta_log_loss,
            color='skyblue')

axes[1,0].errorbar(x = log_loss_mean_stacked[sorted_meta_log_loss],
                 y   = sorted_meta_log_loss,
                 xerr= log_loss_std_stacked[sorted_meta_log_loss],
                 fmt ='o',
                 color='black',
                 capsize=5)

axes[1,0].set_xlabel("Mean Log Loss")
axes[1,0].set_ylabel("Meta Learners")
axes[1,0].set_title("Mean Log Loss of Different Meta Learners")

# Recall Plot

sns.barplot(ax= axes[1,1],
            x = recall_mean_stacked[sorted_meta_recall],
            y = sorted_meta_recall,
            color='lightgreen')

axes[1,1].errorbar(x = recall_mean_stacked[sorted_meta_recall],
                 y   = sorted_meta_recall,
                 xerr= recall_std_stacked[sorted_models_recall],
                 fmt='o', color='black', capsize=5)

axes[1,1].set_xlabel("Mean Recall")
axes[1,1].set_ylabel("Meta Learners")
axes[1,1].set_title("Mean Recall of Different Meta Learners")

plt.tight_layout()
plt.show()


Looking at our results, most individual models operate on a similar level, with our best choices being (in order of descending recall):


| Model      | Log Loss (Mean ± Std) | Recall (Mean ± Std) |
| ---------- | --------------------- | ------------------- |
| Gaussian Naive Bayes (as meta-learner) | 4.68 ± 1.15 | 0.49 ± 0.09 |
| Gradient Boosting | 1.54 ± 0.26 | 0.39 ± 0.05 |
| Logistic Regression | 1.12 ± 0.13 | 0.38 ± 0.05 |
| Support Vector Machine | 1.08 ± 0.06 | 0.35 ± 0.04 |


Since the goal of this project is to identify an illness, maximizing true positives at the expense of disproportionately increasing false positives
can be accepted, so Gaussian Naive Bayes as a meta-learner for our other models might be the best choice depending on preference.
Determining how much importance we put on maximizing true positives over minimizing false positives, we could also check the models' F1 score with
a high beta value to find an "objective" best choice.